# Cluebot

In [124]:
## We first load in packages we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

import xml.etree.ElementTree as ET

In [125]:
tree = ET.parse('/Users/robin/Documents/GitHub/Cluebot/train-edits.xml')
root = tree.getroot()


In [126]:
print(len(root))

for i in range(0, len(root)):
    if root[i][11][2].text != root[i][12][1].text:
        print("!")

15046


In [127]:
edit_id_collection = []
is_vandalism_values = []

for i in range(0, len(root)):
    edit_id_collection.append(int(root[i][1].text))
    if root[i][3].text == "true":
        is_vandalism_values.append(1)
    else:
        is_vandalism_values.append(0)

print(edit_id_collection)
print(is_vandalism_values)

[327231971, 327232052, 327232115, 327232194, 327232220, 327232501, 327232632, 327233017, 327233218, 327233267, 327233416, 327233577, 327233596, 327233598, 327233602, 327233697, 327233819, 327233875, 327233884, 327233987, 327234284, 327234327, 327234633, 327234840, 327234948, 327235026, 327235309, 327235593, 327235970, 327236077, 327236265, 327236357, 327236382, 327236416, 327236534, 327236550, 327236632, 327236641, 327236846, 327237281, 327237289, 327237474, 327237669, 327237891, 327237997, 327238153, 327238336, 327238665, 327238722, 327238842, 327238870, 327239039, 327239241, 327239642, 327239762, 327239803, 327240021, 327240254, 327240417, 327240436, 327240476, 327240504, 327240795, 327240827, 327240897, 327240905, 327240945, 327241278, 327241681, 327241722, 327241791, 327242121, 327242265, 327242290, 327242385, 327242400, 327242497, 327242662, 327242846, 327243221, 327243272, 327243285, 327243433, 327243482, 327243598, 327244105, 327244218, 327244299, 327244465, 327244642, 327244754

In [128]:
# import requests

# url = "https://auth.enterprise.wikimedia.com/v1/login"
# headers = {"Content-Type": "application/json"}
# payload = {
#    "username": "",  # replace with your actual username
#     "password": ""         # replace with your actual password
# }
# response = requests.post(url, json=payload, headers=headers)
# data = response.json()

# print(data.get("id_token"))
# print(data.get("access_token"))
# print(data.get("refresh_token"))
# print(data.get("expires_in"))

In [129]:
import requests

parent_text_collection = []
current_text_collection = []

base_url = "https://en.wikipedia.org/w/api.php"

for i in range(10):
    # Step 1: Get revision and its parent
    params = {
        "action": "query",
        "format": "json",
        "prop": "revisions",
        "revids": edit_id_collection[i],
        "rvprop": "ids|content",
        "rvslots": "main"
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    pages = data['query']['pages']
    page = next(iter(pages.values()))
    revision = page['revisions'][0]

    current_text = revision['slots']['main']['*']
    parent_id = revision['parentid']

    # Step 2: Get parent revision content
    params['revids'] = parent_id
    response2 = requests.get(base_url, params=params)
    data2 = response2.json()

    pages2 = data2['query']['pages']
    parent_page = next(iter(pages2.values()))
    parent_text = parent_page['revisions'][0]['slots']['main']['*']

    # Step 3: Print the before and after
    # print("=== BEFORE (Parent Revision) ===\n")
    # print(parent_text[:1000])  # Trim for readability
    # print("\n=== AFTER (Current Revision) ===\n")
    # print(current_text[:1000])

    parent_text_collection.append(parent_text)
    current_text_collection.append(current_text)

In [130]:
import difflib

vandalism_words_count = {}
constructive_words_count = {}

for i in range(10):
    # Assume you already have these from the revisions:
    # parent_text = text before the edit
    # current_text = text after the edit

    diff = difflib.unified_diff(
        parent_text[i].splitlines(),
        current_text[i].splitlines(),
        fromfile='before',
        tofile='after',
        lineterm=''
    )

    added_lines = []
    deleted_lines = []

    for line in diff:
        if line.startswith('+++') or line.startswith('---') or line.startswith('@@'):
            continue  # skip headers
        elif line.startswith('+'):
            added_lines.append(line[1:])
        elif line.startswith('-'):
            deleted_lines.append(line[1:])

    # Print results
    # print("=== ADDED LINES ===")
    # for line in added_lines:
    #     print(line)

    # print("\n=== DELETED LINES ===")
    # for line in deleted_lines:
    #     print(line)

    if len(added_lines) != 0: 
        words = added_lines[0].split()
    for j in range(len(added_lines)):
        words += added_lines[i].split()
    
    is_vandalism = is_vandalism_values[i]
    for word in words:
        if word in vandalism_words_count:
            if is_vandalism == 1:
                vandalism_words_count[word] += 1
            else:
                constructive_words_count[word] += 1
        else:
            if is_vandalism == 1:
                vandalism_words_count[word] = 1
            else:
                constructive_words_count[word] = 1

print(vandalism_words_count)
print(constructive_words_count)

    

{'==Reception==': 2, 'The': 1, 'game': 1, 'was': 1, 'reviewed': 1, 'in': 3, '1995': 2, "''[[Dragon": 1, "(magazine)|Dragon]]''": 1, '#216': 1, 'by': 1, '[[David': 1, 'Cook': 2, '(game': 2, 'designer)|David': 1, '"Zeb"': 1, 'Cook]]': 1, 'the': 3, '"Eye': 1, 'of': 2, 'Monitor"': 1, 'column.<ref': 1, 'name="Dragon216">{{cite': 1, 'journal': 1, '|title=Eye': 1, 'Monitor': 1, '|author=[[David': 1, 'designer)|Cook,': 1, 'David]]': 1, '|journal=Dragon': 1, '|issue=216': 1, '|date=April': 1, '|pages=63-66}}</ref>': 1}
{'==Reception==': 11, 'The': 6, 'game': 6, 'was': 6, 'reviewed': 6, 'in': 16, '1995': 11, "''[[Dragon": 6, "(magazine)|Dragon]]''": 6, '#216': 6, 'by': 6, '[[David': 6, 'Cook': 11, '(game': 11, 'designer)|David': 6, '"Zeb"': 6, 'Cook]]': 6, 'the': 16, '"Eye': 6, 'of': 11, 'Monitor"': 6, 'column.<ref': 6, 'name="Dragon216">{{cite': 6, 'journal': 6, '|title=Eye': 6, 'Monitor': 6, '|author=[[David': 6, 'designer)|Cook,': 6, 'David]]': 6, '|journal=Dragon': 6, '|issue=216': 6, '|date

In [131]:
words_score = {}
for word in list(vandalism_words_count.keys()):
    if word in constructive_words_count:
        constructive_number = constructive_words_count[word]
    else:
        constructive_number = 0
    words_score[word] = vandalism_words_count[word] / (vandalism_words_count[word] + constructive_number)

print(words_score)

{'==Reception==': 0.15384615384615385, 'The': 0.14285714285714285, 'game': 0.14285714285714285, 'was': 0.14285714285714285, 'reviewed': 0.14285714285714285, 'in': 0.15789473684210525, '1995': 0.15384615384615385, "''[[Dragon": 0.14285714285714285, "(magazine)|Dragon]]''": 0.14285714285714285, '#216': 0.14285714285714285, 'by': 0.14285714285714285, '[[David': 0.14285714285714285, 'Cook': 0.15384615384615385, '(game': 0.15384615384615385, 'designer)|David': 0.14285714285714285, '"Zeb"': 0.14285714285714285, 'Cook]]': 0.14285714285714285, 'the': 0.15789473684210525, '"Eye': 0.14285714285714285, 'of': 0.15384615384615385, 'Monitor"': 0.14285714285714285, 'column.<ref': 0.14285714285714285, 'name="Dragon216">{{cite': 0.14285714285714285, 'journal': 0.14285714285714285, '|title=Eye': 0.14285714285714285, 'Monitor': 0.14285714285714285, '|author=[[David': 0.14285714285714285, 'designer)|Cook,': 0.14285714285714285, 'David]]': 0.14285714285714285, '|journal=Dragon': 0.14285714285714285, '|issu